In [ ]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils import class_weight
import matplotlib.pyplot as plt

# ===================== Parameters =====================
IMG_SIZE = 512
BATCH_SIZE = 32
EPOCHS = 50

# ===================== Load dataset =====================
df = pd.read_csv("dataset.csv")

X, y = [], []
for index, row in df.iterrows():
    img_path = f"./images/{row['image']}"
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue
    old_size = img.shape[:2]
    ratio = float(IMG_SIZE) / max(old_size)
    new_size = tuple([int(x * ratio) for x in old_size])
    img_resized = cv2.resize(img, (new_size[1], new_size[0]))
    delta_w = IMG_SIZE - new_size[1]
    delta_h = IMG_SIZE - new_size[0]
    top, bottom = delta_h // 2, delta_h - delta_h // 2
    left, right = delta_w // 2, delta_w - delta_w // 2
    padded_img = cv2.copyMakeBorder(img_resized, top, bottom, left, right,
                                    cv2.BORDER_CONSTANT, value=0)
    X.append(padded_img)
    y.append(row['tumor'])

X = np.array(X, dtype='float32')
X = preprocess_input(np.repeat(np.expand_dims(X, -1), 3, axis=-1))
y = np.array(y)

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# ===================== Data Augmentation =====================
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
    layers.RandomBrightness(0.1)
])

# ===================== Class weights =====================
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(class_weights))

# ===================== Model =====================
base_model = EfficientNetB3(
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    weights='imagenet'
)

# Unfreeze the last convolutional block (~20 layers)
base_model.trainable = True
for layer in base_model.layers[:-40]:
    layer.trainable = False

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs, outputs)

# ===================== Compile =====================
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')]
)

# ===================== Callbacks =====================
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.5, min_lr=1e-7, verbose=1),
    ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, verbose=1)
]

# ===================== Train =====================
history = model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    class_weight=class_weights,
    callbacks=callbacks
)

# ===================== Save Model =====================
model.save("classification.keras")

# ===================== Evaluate =====================
y_pred = (model.predict(X_val) > 0.5).astype(int)
print("\nClassification Report:\n", classification_report(y_val, y_pred, digits=4))

# ===================== Plot Metrics =====================
def plot_metric(history, metric):
    plt.figure(figsize=(8, 4))
    plt.plot(history.history[metric], label=f'Train {metric}')
    plt.plot(history.history[f'val_{metric}'], label=f'Val {metric}')
    plt.title(f'{metric.capitalize()} over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel(metric.capitalize())
    plt.legend()
    plt.grid(True)
    plt.show()

plot_metric(history, 'accuracy')
plot_metric(history, 'loss')
plot_metric(history, 'precision')
plot_metric(history, 'recall')

: 